# Validate Financial Data Ingestion

This notebook validates the data ingested into the bronze layer table `finance_catalog.bronze.prices`. 
We'll check:
1. Data structure and sample content
2. List of symbols ingested
3. Date range coverage per symbol

## Environment Setup

In [ ]:
# Import required libraries
from pyspark.sql import functions as F
from pyspark.sql.window import Window
import plotly.express as px
import pandas as pd

# Set the catalog
spark.sql("USE CATALOG finance_catalog")
print("Current catalog:", spark.sql("SELECT current_catalog()").collect()[0][0])

## Read Bronze Table Data

Read the ingested data from the bronze layer prices table and create a temporary view for SQL queries.

In [ ]:
# Read the prices table
prices_df = spark.table("bronze.prices")
prices_df.createOrReplaceTempView("prices_view")

# Get basic information about the dataset
print("Total number of records:", prices_df.count())
print("\nSchema:")
prices_df.printSchema()

## Display Sample Data

Show a sample of 10 rows to inspect the data structure and content.

In [ ]:
# Display 10 sample rows ordered by date
display(spark.sql("""
    SELECT *
    FROM prices_view
    ORDER BY date DESC
    LIMIT 10
"""))

## Analyze Distinct Symbols

Get a count of unique symbols and display the list of all tickers in the dataset.

In [ ]:
# Get distinct symbols and their record counts
symbol_counts = spark.sql("""
    SELECT 
        ticker,
        COUNT(*) as record_count
    FROM prices_view
    GROUP BY ticker
    ORDER BY record_count DESC
""")

display(symbol_counts)

## Calculate Date Ranges

Analyze the date range coverage for each symbol to ensure data completeness.

In [ ]:
# Calculate date ranges per symbol
date_ranges = spark.sql("""
    SELECT 
        ticker,
        MIN(date) as start_date,
        MAX(date) as end_date,
        COUNT(DISTINCT date) as trading_days,
        DATEDIFF(MAX(date), MIN(date)) as total_days
    FROM prices_view
    GROUP BY ticker
    ORDER BY ticker
""")

display(date_ranges)

# Calculate any gaps in data
gaps_analysis = spark.sql("""
    WITH dates AS (
        SELECT 
            ticker,
            date,
            LAG(date) OVER (PARTITION BY ticker ORDER BY date) as prev_date
        FROM prices_view
    )
    SELECT 
        ticker,
        date as gap_end,
        prev_date as gap_start,
        DATEDIFF(date, prev_date) as days_gap
    FROM dates
    WHERE DATEDIFF(date, prev_date) > 1
    ORDER BY ticker, date
""")

print("\nChecking for gaps in data:")
display(gaps_analysis)

## Summary

This validation notebook has:
1. Confirmed data structure and content through sample display
2. Identified all unique symbols in the dataset with their record counts
3. Analyzed date ranges for each symbol
4. Detected any gaps in the trading data

Next steps:
- Review any gaps in data to determine if they are holidays or missing data
- Check for any symbols with unexpectedly low record counts
- Verify that adjusted close prices are properly recorded

# Validate Financial Data Ingestion

This notebook validates the data ingested into the bronze layer table `finance_catalog.bronze.prices`. 
We'll check:
1. Data structure and sample content
2. List of symbols ingested
3. Date range coverage per symbol

## Environment Setup

In [ ]:
# Import required libraries
from pyspark.sql import functions as F
from pyspark.sql.window import Window
import plotly.express as px
import pandas as pd

# Set the catalog
spark.sql("USE CATALOG finance_catalog")
print("Current catalog:", spark.sql("SELECT current_catalog()").collect()[0][0])

## Read Bronze Table Data

Read the ingested data from the bronze layer prices table and create a temporary view for SQL queries.

In [ ]:
# Read the prices table
prices_df = spark.table("bronze.prices")
prices_df.createOrReplaceTempView("prices_view")

# Get basic information about the dataset
print("Total number of records:", prices_df.count())
print("\nSchema:")
prices_df.printSchema()

## Display Sample Data

Show a sample of 10 rows to inspect the data structure and content.

In [ ]:
# Display 10 sample rows ordered by date
display(spark.sql("""
    SELECT *
    FROM prices_view
    ORDER BY date DESC
    LIMIT 10
"""))

## Analyze Distinct Symbols

Get a count of unique symbols and display the list of all tickers in the dataset.

In [ ]:
# Get distinct symbols and their record counts
symbol_counts = spark.sql("""
    SELECT 
        ticker,
        COUNT(*) as record_count
    FROM prices_view
    GROUP BY ticker
    ORDER BY record_count DESC
""")

display(symbol_counts)

## Calculate Date Ranges

Analyze the date range coverage for each symbol to ensure data completeness.

In [ ]:
# Calculate date ranges per symbol
date_ranges = spark.sql("""
    SELECT 
        ticker,
        MIN(date) as start_date,
        MAX(date) as end_date,
        COUNT(DISTINCT date) as trading_days,
        DATEDIFF(MAX(date), MIN(date)) as total_days
    FROM prices_view
    GROUP BY ticker
    ORDER BY ticker
""")

display(date_ranges)

# Calculate any gaps in data
gaps_analysis = spark.sql("""
    WITH dates AS (
        SELECT 
            ticker,
            date,
            LAG(date) OVER (PARTITION BY ticker ORDER BY date) as prev_date
        FROM prices_view
    )
    SELECT 
        ticker,
        date as gap_end,
        prev_date as gap_start,
        DATEDIFF(date, prev_date) as days_gap
    FROM dates
    WHERE DATEDIFF(date, prev_date) > 1
    ORDER BY ticker, date
""")

print("\nChecking for gaps in data:")
display(gaps_analysis)

## Summary

This validation notebook has:
1. Confirmed data structure and content through sample display
2. Identified all unique symbols in the dataset with their record counts
3. Analyzed date ranges for each symbol
4. Detected any gaps in the trading data

Next steps:
- Review any gaps in data to determine if they are holidays or missing data
- Check for any symbols with unexpectedly low record counts
- Verify that adjusted close prices are properly recorded